## Image reconstruction using Maximum Likeliood Expectation Maximization (MLEM)


Maximum Likeliood Expectation Maximization (MLEM) is one of the most used iterative image reconstruction algorithms used to reconstruct PET and SPECT images. Detailed background information on this algorithm for PET and SPECT image reconstruction is available in [this video](https://www.youtube.com/watch?v=CHKOSYdf47c) and in [this video](https://www.youtube.com/watch?v=Z70n5NCw9BY). Moreover, background information on the concept of maximum likelihood approaches are available [here](https://www.youtube.com/watch?v=uTa7g_h4c1E). Background information on PET and SPECT is available [here](https://www.youtube.com/watch?v=M8DOzE2d0dw) and [here](https://www.youtube.com/watch?v=4mrtq8CeLvo&list=PLKkWkQgtnBS1tWAE3-TL1-MDKY9EUJTFP&index=2).

### Learning objective

The aim of this notebook is to implement the iterative MLEM for reconstruction of simulated 2D PET data.

### The MLEM update algorithm

Since MLEM is an iterative algorithm, it tells us how to calculated an updated image $x^{n+1}$ based on a current image $x^n$ and measured PET emission data $y$ via

\begin{equation}
x^{n+1} = \frac{x^n}{P^T \mathbb{1}} P^T \left( \frac{y}{\bar{y}(x^n)} \right) \ ,
\label{eq:MLEM_update}
\tag{1}
\end{equation}

where

\begin{equation}
\bar{y}(x^n) = Px^n + s \ ,
\label{eq:fwd_model}
\tag{2}
\end{equation}

Eq. (2) is the so-called forward model that calculates which data $\bar{y}$ to expect based on the current image $x^n$. The linear operator $P$ includes a model of the PET acquisition physics (e.g. calculation of line integrals, and correction for attenuation and detector sensitivities). Moreover, $s$ denotes the contribution of random and scattered coincidences. In many books, the $P$ is called the *forward model* and the application of $P$ to $x^n$ is called the *(corrected) forward projection* of $x^n$ which maps from image space into the data (sinogram) space.
Accordingly, the adjoint (transposed) operator $P^T$ - often also called the back projection - maps from data (sinogram) space into image space. Note that (i) $\mathbb{1}$ is a data set (sinogram) full of ones and (ii) the divisions in (1) are to be understood point-wise. 

**In this notebook, we assume that $P$, $P^T$, $s$ are known and we will provide simplistic 2D implementations of them.**

### Module import section

In [ ]:
# import of modules that we need in this notebook
# make sure that utils.py is placed in the same directory 

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from utils import RotationBased2DProjector, PETAcquisitionModel, test_images

# needed to get inline matplotlib plots in an IPython notebook
%matplotlib inline

### Input parameter section

In [ ]:
# input parameter for our demo

# number of pixels for our images to be simulated and reconstructed
npix = 100
# pixel size in mm
pix_size_mm = 4

plt.rcParams['image.cmap'] = 'Greys'
np.random.seed(1)

### Image setup

In [ ]:
# generate the ground truth activity (emission) and attenuation images that we use for the data simulation
em_img, att_img = test_images(npix)

In [ ]:
# show the true activity (emission) image and the attenuation image
fig, ax = plt.subplots(1,2)
im0 = ax[0].imshow(em_img)
im1 = ax[1].imshow(att_img)
fig.colorbar(im0, ax = ax[0], location = 'bottom')
fig.colorbar(im1, ax = ax[1], location = 'bottom')
fig.tight_layout()

print(f'image shape {em_img.shape}')

### Data simulation 

In [ ]:
# setup the forward projector"
# the acq_model object is an abstract representation of the linear operator P (and also it's adjoint)
proj = RotationBased2DProjector(npix, pix_size_mm = pix_size_mm, num_subsets = 1)
acq_model = PETAcquisitionModel(proj, att_img)

In [ ]:
# generate noise free data by applying the acquisition model to our simulated emission image
noise_free_data = acq_model.forward(em_img)

In [ ]:
# add poisson noise to the data
noisy_data = np.random.poisson(noise_free_data)

In [ ]:
# show the noise-free and noisy simulated emission data (sinogram)
fig2, ax2 = plt.subplots(1,2)
im02 = ax2[0].imshow(noise_free_data)
im12 = ax2[1].imshow(noisy_data)
fig2.colorbar(im02, ax = ax2[0], location = 'bottom')
fig2.colorbar(im12, ax = ax2[1], location = 'bottom')
ax2[0].set_xlabel('radial element')
ax2[1].set_xlabel('radial element')
ax2[0].set_ylabel('view')
ax2[1].set_ylabel('view')
fig2.tight_layout()

print(f'data (sinogram) shape {noise_free_data.shape}')